In [ ]:


class Application(fix.Application):
    def onCreate(self, sessionID):
        print(f"Session created: {sessionID}")

    def onLogon(self, sessionID):
        print(f"Successfully logged in: {sessionID}")

    def onLogout(self, sessionID):
        print(f"Successfully logged out: {sessionID}")

    def toAdmin(self, message, sessionID):
        print(f"Sending admin message: {message}")

    def fromAdmin(self, message, sessionID):
        print(f"Received admin message: {message}")

    def toApp(self, message, sessionID):
        print(f"Sending app message: {message}")

    def fromApp(self, message, sessionID):
        print(f"Received app message: {message}")
        self.onMessage(message, sessionID)

    def onMessage(self, message, sessionID):
        print(f"Message from {sessionID}: {message}")

    def send_order(self, sessionID):
        # 创建一个新的市场订单（买入 100股 MSFT）
        order = fix.Message()
        order.getHeader().setField(fix.MsgType(fix.MsgType_NewOrderSingle))
        order.setField(fix.ClOrdID("12345"))    # 唯一订单ID
        order.setField(fix.HandlInst(fix.HandlInst_MANUAL_ORDER_BEST_EXECUTION))
        order.setField(fix.Symbol("MSFT"))      # 股票代码
        order.setField(fix.Side(fix.Side_BUY))  # 买入
        order.setField(fix.TransactTime(int(time.time())))
        order.setField(fix.OrdType(fix.OrdType_MARKET))  # 市场单
        order.setField(fix.OrderQty(100))  # 购买数量

        fix.Session.sendToTarget(order, sessionID)
        print("Order sent!")


def main():
    # 创建配置和应用程序实例
    settings = fix.SessionSettings("client.cfg")
    application = Application()
    storeFactory = fix.FileStoreFactory(settings)
    logFactory = fix.FileLogFactory(settings)
    initiator = fix.SocketInitiator(application, storeFactory, settings, logFactory)

    # 启动连接
    initiator.start()
    print("Connection started...")

    # 等待登录完成
    time.sleep(5)

    # 获取会话ID，并发送订单
    sessionID = settings.getSessions()[0]
    application.send_order(sessionID)

    # 等待执行
    time.sleep(10)

    # 断开连接
    initiator.stop()
    print("Connection stopped.")


if __name__ == "__main__":
    main()
